# Variaveis Pagamento

In [0]:
-- Databricks notebook source
WITH tb_pedidos AS (

  SELECT 
      DISTINCT 
      t1.idPedido,
      t2.idVendedor

  FROM silver.olist.pedido AS t1

  LEFT JOIN silver.olist.item_pedido as t2
  ON t1.idPedido = t2.idPedido

  WHERE t1.dtPedido < '2018-01-01'
  AND t1.dtPedido >= add_months('2018-01-01', -6)
  AND idVendedor IS NOT NULL

),

tb_join AS (

  SELECT 
        t1.idVendedor,
        t2.*         

  FROM tb_pedidos AS t1

  LEFT JOIN silver.olist.pagamento_pedido AS t2
  ON t1.idPedido = t2.idPedido

),

tb_group AS (

  SELECT idVendedor,
         descTipoPagamento,
         count(distinct idPedido) as qtdePedidoMeioPagamento,
         sum(vlPagamento) as vlPedidoMeioPagamento

  FROM tb_join

  GROUP BY idVendedor, descTipoPagamento
  ORDER BY idVendedor, descTipoPagamento

 ), tb_summary as (
   
SELECT 
  idVendedor,

  sum(case when descTipoPagamento='boleto' then qtdePedidoMeioPagamento else 0 end) as qtde_boleto_pedido,
  sum(case when descTipoPagamento='credit_card' then qtdePedidoMeioPagamento else 0 end) as qtde_credit_card_pedido,
  sum(case when descTipoPagamento='voucher' then qtdePedidoMeioPagamento else 0 end) as qtde_voucher_pedido,
  sum(case when descTipoPagamento='debit_card' then qtdePedidoMeioPagamento else 0 end) as qtde_debit_card_pedido,

  sum(case when descTipoPagamento='boleto' then vlPedidoMeioPagamento else 0 end) as valor_boleto_pedido,
  sum(case when descTipoPagamento='credit_card' then vlPedidoMeioPagamento else 0 end) as valor_credit_card_pedido,
  sum(case when descTipoPagamento='voucher' then vlPedidoMeioPagamento else 0 end) as valor_voucher_pedido,
  sum(case when descTipoPagamento='debit_card' then vlPedidoMeioPagamento else 0 end) as valor_debit_card_pedido,
  
  sum(case when descTipoPagamento='boleto' then qtdePedidoMeioPagamento else 0 end) / sum(qtdePedidoMeioPagamento) as pct_qtd_boleto_pedido,
  sum(case when descTipoPagamento='credit_card' then qtdePedidoMeioPagamento else 0 end) / sum(qtdePedidoMeioPagamento) as pct_qtd_credit_card_pedido,
  sum(case when descTipoPagamento='voucher' then qtdePedidoMeioPagamento else 0 end) / sum(qtdePedidoMeioPagamento) as pct_qtd_voucher_pedido,
  sum(case when descTipoPagamento='debit_card' then qtdePedidoMeioPagamento else 0 end) / sum(qtdePedidoMeioPagamento) as pct_qtd_debit_card_pedido,

  sum(case when descTipoPagamento='boleto' then vlPedidoMeioPagamento else 0 end) / sum(vlPedidoMeioPagamento) as pct_valor_boleto_pedido,
  sum(case when descTipoPagamento='credit_card' then vlPedidoMeioPagamento else 0 end) / sum(vlPedidoMeioPagamento) as pct_valor_credit_card_pedido,
  sum(case when descTipoPagamento='voucher' then vlPedidoMeioPagamento else 0 end) / sum(vlPedidoMeioPagamento) as pct_valor_voucher_pedido,
  sum(case when descTipoPagamento='debit_card' then vlPedidoMeioPagamento else 0 end) / sum(vlPedidoMeioPagamento) as pct_valor_debit_card_pedido

FROM tb_group

GROUP BY idVendedor
 ), tb_cartao as (
   select 
   idVendedor, 
   avg(nrParcelas) as avgParcelas,
   PERCENTILE(nrParcelas, 0.5) as mediaQtParcelas,
   max(nrParcelas) as maxQtdeParcelas,
   min(nrParcelas) as minQtdeParcelas
 from tb_join
 where descTipoPagamento = 'credit_card'
 group by idVendedor
 )
 
 select 
  '2018-01-01' as dtReference,
  t1.*,
  t2.avgParcelas,
  t2.mediaQtParcelas,
  t2.maxQtdeParcelas,
  t2.minQtdeParcelas
from tb_summary as t1
 LEFT JOIN tb_cartao as t2
 ON t1.idVendedor = t2.idVendedor


# Variaveis cliente

In [0]:
with tb_join as (
  SELECT
    t1.idPedido,
    t1.idCliente,
    t2.idVendedor,
    t3.descUF
  FROM
    silver.olist.pedido AS t1
    LEFT JOIN silver.olist.item_pedido AS t2 ON t1.idPedido = t2.idPedido
    LEFT JOIN silver.olist.cliente as t3 on t1.idCliente = t3.idCliente
  WHERE
    dtPedido < '2018-01-01'
    AND dtPedido >= add_months('2018-01-01', -6)
    AND idVendedor IS NOT NULL
), tb_group as (

select
  idVendedor,
  count(distinct descUF) as qtdeUFsPedidos,
  count(distinct case when descUF = 'AC' then idPedido end) / count(distinct idPedido) as pctPedidoAC,
  count(distinct case when descUF = 'AL' then idPedido end) / count(distinct idPedido) as pctPedidoAL,
  count(distinct case when descUF = 'AM' then idPedido end) / count(distinct idPedido) as pctPedidoAM,
  count(distinct case when descUF = 'AP' then idPedido end) / count(distinct idPedido) as pctPedidoAP,
  count(distinct case when descUF = 'BA' then idPedido end) / count(distinct idPedido) as pctPedidoBA,
  count(distinct case when descUF = 'CE' then idPedido end) / count(distinct idPedido) as pctPedidoCE,
  count(distinct case when descUF = 'DF' then idPedido end) / count(distinct idPedido) as pctPedidoDF,
  count(distinct case when descUF = 'ES' then idPedido end) / count(distinct idPedido) as pctPedidoES,
  count(distinct case when descUF = 'GO' then idPedido end) / count(distinct idPedido) as pctPedidoGO,
  count(distinct case when descUF = 'MA' then idPedido end) / count(distinct idPedido) as pctPedidoMA,
  count(distinct case when descUF = 'MG' then idPedido end) / count(distinct idPedido) as pctPedidoMG,
  count(distinct case when descUF = 'MS' then idPedido end) / count(distinct idPedido) as pctPedidoMS,
  count(distinct case when descUF = 'MT' then idPedido end) / count(distinct idPedido) as pctPedidoMT,
  count(distinct case when descUF = 'PA' then idPedido end) / count(distinct idPedido) as pctPedidoPA,
  count(distinct case when descUF = 'PB' then idPedido end) / count(distinct idPedido) as pctPedidoPB,
  count(distinct case when descUF = 'PE' then idPedido end) / count(distinct idPedido) as pctPedidoPE,
  count(distinct case when descUF = 'PI' then idPedido end) / count(distinct idPedido) as pctPedidoPI,
  count(distinct case when descUF = 'PR' then idPedido end) / count(distinct idPedido) as pctPedidoPR,
  count(distinct case when descUF = 'RJ' then idPedido end) / count(distinct idPedido) as pctPedidoRJ,
  count(distinct case when descUF = 'RN' then idPedido end) / count(distinct idPedido) as pctPedidoRN,
  count(distinct case when descUF = 'RO' then idPedido end) / count(distinct idPedido) as pctPedidoRO,
  count(distinct case when descUF = 'RR' then idPedido end) / count(distinct idPedido) as pctPedidoRR,
  count(distinct case when descUF = 'RS' then idPedido end) / count(distinct idPedido) as pctPedidoRS,
  count(distinct case when descUF = 'SC' then idPedido end) / count(distinct idPedido) as pctPedidoSC,
  count(distinct case when descUF = 'SE' then idPedido end) / count(distinct idPedido) as pctPedidoSE,
  count(distinct case when descUF = 'SP' then idPedido end) / count(distinct idPedido) as pctPedidoSP,
  count(distinct case when descUF = 'TO' then idPedido end) / count(distinct idPedido) as pctPedidoTO
from
  tb_join
group by idVendedor
)
select 
'2018-01-01' as dtReference,
* from tb_group

# Variaveis de produto

In [0]:
-- Databricks notebook source
WITH tb_join AS (

  SELECT DISTINCT
         t2.idVendedor,
         t3.*

  FROM silver.olist.pedido AS t1

  LEFT JOIN silver.olist.item_pedido AS t2
  ON t1.idPedido = t2.idPedido

  LEFT JOIN silver.olist.produto as t3
  ON t2.idProduto = t3.idProduto

  WHERE t1.dtPedido < '2018-01-01'
  AND t1.dtPedido >= add_months('2018-01-01', -6)
  AND t2.idVendedor IS NOT NULL

),

tb_summary as (

  SELECT idVendedor,
         avg(coalesce(nrFotos,0)) as avgFotos,
         avg(vlComprimentoCm * vlAlturaCm * vlLarguraCm) as avgVolumeProduto,
         percentile(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0.5) as medianVolumeProduto,
         min(vlComprimentoCm * vlAlturaCm * vlLarguraCm) as minVolumeProduto,
         max(vlComprimentoCm * vlAlturaCm * vlLarguraCm) as maxVolumeProduto,
         
        count(distinct case when descCategoria = 'cama_mesa_banho' then idProduto end) / count(distinct idProduto) as pctCategoriacama_mesa_banho,
        count(distinct case when descCategoria = 'beleza_saude' then idProduto end) / count(distinct idProduto) as pctCategoriabeleza_saude,
        count(distinct case when descCategoria = 'esporte_lazer' then idProduto end) / count(distinct idProduto) as pctCategoriaesporte_lazer,
        count(distinct case when descCategoria = 'informatica_acessorios' then idProduto end) / count(distinct idProduto) as pctCategoriainformatica_acessorios,
        count(distinct case when descCategoria = 'moveis_decoracao' then idProduto end) / count(distinct idProduto) as pctCategoriamoveis_decoracao,
        count(distinct case when descCategoria = 'utilidades_domesticas' then idProduto end) / count(distinct idProduto) as pctCategoriautilidades_domesticas,
        count(distinct case when descCategoria = 'relogios_presentes' then idProduto end) / count(distinct idProduto) as pctCategoriarelogios_presentes,
        count(distinct case when descCategoria = 'telefonia' then idProduto end) / count(distinct idProduto) as pctCategoriatelefonia,
        count(distinct case when descCategoria = 'automotivo' then idProduto end) / count(distinct idProduto) as pctCategoriaautomotivo,
        count(distinct case when descCategoria = 'brinquedos' then idProduto end) / count(distinct idProduto) as pctCategoriabrinquedos,
        count(distinct case when descCategoria = 'cool_stuff' then idProduto end) / count(distinct idProduto) as pctCategoriacool_stuff,
        count(distinct case when descCategoria = 'ferramentas_jardim' then idProduto end) / count(distinct idProduto) as pctCategoriaferramentas_jardim,
        count(distinct case when descCategoria = 'perfumaria' then idProduto end) / count(distinct idProduto) as pctCategoriaperfumaria,
        count(distinct case when descCategoria = 'bebes' then idProduto end) / count(distinct idProduto) as pctCategoriabebes,
        count(distinct case when descCategoria = 'eletronicos' then idProduto end) / count(distinct idProduto) as pctCategoriaeletronicos

  FROM tb_join

  GROUP BY idVendedor

)

SELECT '2018-01-01' AS dtReference,
       *

FROM tb_summary

In [0]:
SELECT
  descCategoria
from
  silver.olist.item_pedido as t2
  LEFT JOIN silver.olist.produto as t3 ON t2.idProduto = t3.idProduto
WHERE
  t2.idVendedor IS NOT NULL
GROUP BY 1
ORDER BY 
  count(descCategoria) DESC
LIMIT 15

# Variaveis Avaliacao

In [0]:
WITH tb_pedido as (
  SELECT
    DISTINCT t1.idPedido,
    t2.idVendedor
  FROM
    silver.olist.pedido AS t1
    LEFT JOIN silver.olist.item_pedido AS t2 ON t1.idPedido = t2.idPedido
  WHERE
    t1.dtPedido < '2018-01-01'
    AND t1.dtPedido >= add_months('2018-01-01', -6)
    AND t2.idVendedor IS NOT NULL
),
tb_join as (
  SELECT
    t1.*,
    t2.vlNota
  FROM
    tb_pedido t1
    LEFT JOIN silver.olist.avaliacao_pedido t2 ON t1.idPedido = t2.idPedido
), tb_summary as (

select
  idVendedor,
  avg(vlNota) as avgNota,
  percentile(vlNota, 0.5) as medianNota,
  min(vlNota) as minNota,
  max(vlNota) as maxNota,
  count(vlNota) / count(idPedido) as pctAvaliacao
from
  tb_join
group by 1
)
select '2018-01-01' as dtReference, t1.* from tb_summary as t1

# Variaveis de entrega

In [0]:
WITH tb_pedido as (
  SELECT
    t1.idPedido,
    t2.idVendedor,
    t1.descSituacao,
    t1.dtPedido,
    t1.dtAprovado,
    t1.dtEntregue,
    t1.dtEstimativaEntrega,
    sum(vlFrete) as totalFrete
  FROM
    silver.olist.pedido t1
    LEFT JOIN silver.olist.item_pedido as t2 on t1.idPedido = t2.idPedido
  WHERE
    t1.dtPedido < '2018-01-01'
    AND t1.dtPedido >= add_months('2018-01-01', -6) --     AND t2.idVendedor IS NOT NULL
  GROUP BY
    t1.idPedido,
    t2.idVendedor,
    t1.descSituacao,
    t1.dtPedido,
    t1.dtAprovado,
    t1.dtEntregue,
    t1.dtEstimativaEntrega
)
SELECT
  idVendedor,
  COUNT(
    DISTINCT CASE
      WHEN  DATE(coalesce(dtEntregue, '2018-01-01')) > DATE(dtEstimativaEntrega) THEN idPedido
    END
  ) / count(
    distinct CASE
      WHEN descSituacao = 'delivered' then idPedido
    END
  ) as pctPedidoCancelado,
  count(distinct case when descSituacao = 'canceled' then idPedido end) / count(distinct idPedido) as pctPedidoCancelado,
  avg(totalFrete) as avgFrete,
  percentile(totalFrete, 0.5) medianFrete,
  max(totalFrete) as maxFrete,
  min(totalFrete) as minFrete,
  avg(datediff(coalesce(dtEntregue, '2018-01-01'), dtAprovado)) as qtdDiasAprovadoEngrega,
  avg(datediff(coalesce(dtEntregue, '2018-01-01'), dtPedido)) as qtdDiasAprovadoEngrega,
  avg(datediff(coalesce(dtEntregue, '2018-01-01'), dtEstimativaEntrega)) as qtdeDiasEntregaPromessa
FROM
  tb_pedido
group by
  1